In [1]:
from bs4 import BeautifulSoup
from requests import TooManyRedirects
import re
import requests
from datetime import datetime
import os
import pandas as pd

from scrape_util import *
from scrape_function import *

In [2]:
def get_url_response(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('Aborted as response code is not 200')
            return None
    except error as e:
        print(f'[request {url} failed] : {e}')
        return None
    return json.loads(response.content)

In [3]:
def get_value_from_dict_key(keys,d):
    r = []
    for k in keys:
        if k in d.keys():
            r.append(d[k])
        else:
            r.append(None)
    return r

In [4]:
def get_movie_name(sffx, limit = None, limit_sec = 60*60*30, start_token = None, dir = ''):
    '''
    sffx: the suffix of request url used to specified which data is going to be requested
    limit: number of requested pages limit
    limit_sec: number of request second limit
    start_token: the most recent token (used in continue scrape case)
    dir: specific for directory
    '''
    begin_time = datetime.now()
    # create dataframe
    orgn_sffx = sffx
    df = pd.DataFrame({'title':[],
                     'date_list':[],
                     'critic_sentiment':[],
                     'critic_score':[],
                     'audience_setiment':[],
                     'audience_score':[],
                     'media_url':[]
                       
                    })
    # first page
    is_first_iter = True
    # response = get_url_response(url)
    def append_to_df(df, value_to_append):
        df_to_append = pd.DataFrame({'title':[value_to_append[0]],
                                     'date_list':[value_to_append[1]],
                                     'critic_sentiment':[value_to_append[2]],
                                     'critic_score':[value_to_append[3]],
                                     'audience_setiment':[value_to_append[4]],
                                     'audience_score':[value_to_append[5]],
                                     'media_url':[value_to_append[6]]
                                    })
        return df._append(df_to_append, ignore_index = True)
    # following page
    # Initial start_key
    if start_token is None:
        count = 1
        startCursor = 'no cursor'
    elif start_token == 'Mjc':
        count = 2
    else:
        count = 3
        startCursor = start_token
    init_count = count
    print(f'init_count = {init_count}')
    while True:
        if limit is not None:
            if count > limit:
                break
        
        if count == 1:
            sffx = orgn_sffx
        elif count == 2:
            startCursor = 'Mjc'
            sffx = f'{orgn_sffx}?after={startCursor}%3D'
        else:
            sffx = f'{orgn_sffx}?after={startCursor}%3D'
        url = f'https://www.rottentomatoes.com/napi/browse/{sffx}'
        print(f'Try to get response from url {url}')
        response = get_url_response(url)
        
        # skip for unsuccessful request
        if not response:
            continue
        
        for movie in response['grid']['list']:
            title, dt, critics, audiences, media_url = get_value_from_dict_key(keys = ['title','releaseDateText','criticsScore', 'audienceScore','mediaUrl']
                                                         ,d = movie
                                                        )
            critic_sentiment, critic_score = get_value_from_dict_key(keys = ['sentiment','scorePercent'], d = critics)
            audience_setiment, audience_score = get_value_from_dict_key(keys = ['sentiment','scorePercent'], d = audiences)
            value_to_append = title, dt, critic_sentiment, critic_score, audience_setiment, audience_score, media_url
            if not start_token == startCursor:
                df = append_to_df(df,value_to_append)

        if not 'pageInfo' in response.keys():
            break
        else:
            if not response['pageInfo']['hasNextPage']:
                return df,'end_of_scrape'
                break
            else:
                startCursor = response['pageInfo']['endCursor'].replace('=','')
        count += 1

        # stop if reach limited time
        if (datetime.now() - begin_time).seconds > limit_sec:
            break
    print('saving scrape result..........')
    df.to_csv(f'{dir}/{orgn_sffx}_strtoken{start_token}_stptoken{startCursor}_l{limit}_tl{limit_sec}.csv', index = False)
                
                                     
                                     
    
    return df,startCursor

In [5]:
# _ , token = get_movie_name(f'movies_at_home',limit = 5000, limit_sec = 60*2, start_token = None,dir = 'movies_at_home')

init_count = 1
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=Mjc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=NTc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=ODc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTE3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTQ3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTc3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MjA3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MjM3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=

NameError: name 'error' is not defined

In [ ]:
# _ , token = get_movie_name(f'movies_at_home',limit = 5000, limit_sec = 60*2, start_token = token,dir = 'movies_at_home')

In [6]:
is_first_iter = True
while True:
    if is_first_iter:
        token = None
    _ , token = get_movie_name(f'movies_at_home',limit = 5000, limit_sec = 60*2, start_token = token,dir = 'movies_at_home')
    is_first_iter = False
    if token == 'end_of_scrape':
        break

init_count = 1
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=Mjc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=NTc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=ODc%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTE3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTQ3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MTc3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MjA3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=MjM3%3D
Try to get response from url https://www.rottentomatoes.com/napi/browse/movies_at_home?after=

# unused

In [ ]:
def get_movie_list_init_page(url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/sort:newest'):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    title_list = [s.contents[0].strip() for s in soup.find_all('span', {'data-qa': 'discovery-media-list-item-title'})]
    date_list = [s.contents[0].strip() for s in soup.find_all('span', {'data-qa': 'discovery-media-list-item-start-date'})]
    date_list = [datetime.strptime(' '.join(x.split()[1:]), '%b %d, %Y') for x in date_list]
    critic_sentiment = [x.attrs['sentiment'] for x in soup.find_all('score-icon-critics',{'slot': 'criticsScoreIcon'})]
    critic_score = [x.contents[0].strip() for x in soup.find_all('rt-text',{'slot': 'criticsScore'})]
    audience_setiment = [x.attrs['sentiment'] for x in soup.find_all('score-icon-audience',{'slot': 'audienceScoreIcon'})]
    audience_score = [x.contents[0].strip() for x in soup.find_all('rt-text',{'slot': 'audienceScore'})]

    l_data = title_list,critic_sentiment,critic_score,audience_setiment,audience_score
    l_data = map(map_empty_string_to_null,l_data)
    
    data = list(zip(*map(map_empty_string_to_null,[title_list, date_list, critic_sentiment, critic_score, audience_setiment, audience_score])))

    return pd.DataFrame(data, columns = ['title_list','date_list','critic_sentiment', 'critic_score', 'audience_setiment', 'audience_score',])